## Homework

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and model management.

### Q1. Install MLFlow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

Ans: **2.17.0**

In [9]:
!mlflow --version

mlflow, version 2.17.0


### Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

- load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
- fit a DictVectorizer on the training set (January 2023 data),
- save the preprocessed datasets and the DictVectorizer to disk.

Your task is to download the datasets and then execute this command:

```python
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to OUTPUT_FOLDER?

Ans: **4**

In [10]:
import pandas as pd

In [11]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -P ./data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet -P ./data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet -P ./data

--2024-10-27 02:36:28--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.160, 3.164.82.40, 3.164.82.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘./data/green_tripdata_2023-01.parquet.1’

green_tripdata_2023 100%[===================>]   1.36M  1.82MB/s    in 0.7s    

2024-10-27 02:36:30 (1.82 MB/s) - ‘./data/green_tripdata_2023-01.parquet.1’ saved [1427002/1427002]

--2024-10-27 02:36:30--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.112, 3.164.82.160, 3.164.82.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.112|:443... connected.
HTTP re

In [12]:
!python ./scripts/preprocess_data.py --raw_data_path ./data --dest_path ./output

In [13]:
!ls -lh ./output

total 6.9M
-rw-rw-rw- 1 codespace codespace 128K Oct 27 02:36 dv.pkl
-rw-rw-rw- 1 codespace codespace 2.4M Oct 27 02:36 test.pkl
-rw-rw-rw- 1 codespace codespace 2.3M Oct 27 02:36 train.pkl
-rw-rw-rw- 1 codespace codespace 2.2M Oct 27 02:36 val.pkl


### Q3. Train a model with autolog

We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

-load the datasets produced by the previous step,
-train the model on the training set,
-calculate the RMSE score on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the min_samples_split parameter:

Ans: **2**

In [8]:
import mlflow

TRACKING_URI="http://127.0.0.1:5001"
EXPERIMENT_NAME="nyc-green-taxi"

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/workspaces/mlops-zoomcamp-hws/02-experiment tracking/artificats/1', creation_time=1729994246907, experiment_id='1', last_update_time=1729994246907, lifecycle_stage='active', name='nyc-green-taxi', tags={}>

In [4]:
!pygmentize -g ./scripts/train.py

import os
import pickle
import click

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)
    

def train_model(data_path: str):
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)

    return rf, rmse


@click.command()
@click.option(
    "--data_path",
    default="./output",
    help="Location where the processed NYC taxi trip data was saved"
)
def run_train(data_path: str):
    train_model(data_path)
    


if __name__ == '__main__':
    run_train()


In [3]:
import mlflow
from scripts.train import train_model

mlflow.autolog()

with mlflow.start_run():
    train_model(data_path='./output')

2024/10/27 02:44:29 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


2024/10/27 02:44:29 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/workspaces/mlops-zoomcamp-hws/02-experiment tracking/venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/10/27 02:44:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run redolent-bee-273 at: http://127.0.0.1:5001/#/experiments/1/runs/c13f613ed7b642538434b4900b3c08bb.
2024/10/27 02:44:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5001/#/experiments/1.


In [9]:
from mlflow.tracking import MlflowClient

client = MlflowClient(TRACKING_URI)

def print_experiment(experiment):
    print(f"(Experiment)")
    print(f"    experiment_id={experiment.experiment_id}")
    print(f"    name='{experiment.name}'")
    print(f"    artifact_location='{experiment.artifact_location}'")
    print()

for experiment in client.search_experiments():
    print_experiment(experiment)

(Experiment)
    experiment_id=2
    name='random-forest-hyperopt'
    artifact_location='/workspaces/mlops-zoomcamp-hws/02-experiment tracking/artifacts/2'

(Experiment)
    experiment_id=1
    name='nyc-green-taxi'
    artifact_location='/workspaces/mlops-zoomcamp-hws/02-experiment tracking/artificats/1'

(Experiment)
    experiment_id=0
    name='Default'
    artifact_location='/workspaces/mlops-zoomcamp-hws/02-experiment tracking/artificats/0'



In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=1,
    filter_string='',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse_valid ASC"]
)

for run in runs:
    info = client.get_run(run.info.run_id)

    print(f"Run ID: {run.info.run_id}")
    print("Parameters:")

    for param_key, param_value in info.data.params.items():
        print(f"\t{param_key}: {param_value}")

    print()

Run ID: c13f613ed7b642538434b4900b3c08bb
Parameters:
	bootstrap: True
	ccp_alpha: 0.0
	criterion: squared_error
	max_depth: 10
	max_features: 1.0
	max_leaf_nodes: None
	max_samples: None
	min_impurity_decrease: 0.0
	min_samples_leaf: 1
	min_samples_split: 2
	min_weight_fraction_leaf: 0.0
	monotonic_cst: None
	n_estimators: 100
	n_jobs: None
	oob_score: False
	random_state: 0
	verbose: 0
	warm_start: False

Run ID: 70b91497d4414d498961b61819eacb29
Parameters:

Run ID: 44023533f8194ff6b1a898f436edee64
Parameters:



### Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:
- launch the tracking server on your local machine,
- select a SQLite db for the backend store and a folder called artifacts for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

Ans: **default-artifact-root**

### Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using hyperopt. We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:
- the list of hyperparameters that are passed to the objective function during the optimization,
- the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

Ans: **5.335**

In [2]:
!pygmentize -g ./scripts/hpo.py

import os
import pickle
import click
import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


@click.command()
@click.option(
    "--data_path",
    default="./output",
    help="Location where the processed NYC taxi trip data was saved"
)
@click.option(
    "--num_trials",
    default=15,
    help="The number of parameter evaluations for the optimizer to explore"
)
def run_optimization(data_path: str, num_trials: int):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    def objective(params):
        with mlflow.start_run():
        

In [4]:
!python ./scripts/hpo.py

2024/10/27 14:16:36 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/workspaces/mlops-zoomcamp-hws/02-experiment tracking/venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/10/27 14:16:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run agreeable-seal-151 at: http://127.0.0.1:5001/#/experiments/2/runs/ea1106b339e54bffa9d72cd1d5b5827e.

2024/10/27 14:16:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5001/#/experiments/2.

  7%|▋          | 1/15 [00:07<01:47,  7.64s/trial, best loss: 5.370086069268862]/workspaces/mlops-zoomcamp-hws/02-experiment tracking/venv/lib/python3.9/site-packages

In [11]:
runs = client.search_runs(
    experiment_ids=2,
    filter_string='',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=13,
    order_by=["metrics.rmse_valid ASC"]
)

for run in runs:
    name = run.info.run_name

    format_name = name + " " * (16 - len(name)) if len(name) <= 16 else name[:13] + "..."
    print(f"{format_name} {run.info.run_id} rmse: {run.data.metrics['rmse']:.3f}")

handsome-carp... 5364fe187c9d4ed3a042da15c0f7e417 rmse: 5.355
gentle-owl-482   7d20b282b99b4ea097a91e0505bdcb39 rmse: 5.372
awesome-skunk... e23340f0ebe4426196b24a756e616688 rmse: 5.538
gregarious-ee... ab7d9503242f40ed9dd033d7de881e85 rmse: 5.484
angry-shark-732  c8be20d68f93459b9f7e1abe68264df0 rmse: 5.335
bold-cat-636     687498e62d814599ade2cd820f7a49ea rmse: 5.363
monumental-mo... 52954a0496fc4c2e8d2d650d2f059c65 rmse: 5.411
enchanting-gr... c3b0d2b40fc14f659785720c73ae2b3e rmse: 5.465
painted-goose... ebf1e50398424a41bcb7c10c60a2297d rmse: 5.364
bouncy-horse-467 411569a0b1ad4e4692b91c2e46025ad4 rmse: 5.355
rebellious-pa... 26404ca8549e4c099e5b9b7520e5866d rmse: 5.443
sincere-dolph... 3b104a6d43e145f0a01978695fea89f7 rmse: 5.357
youthful-ape-344 a541df84850f4b3698c68b5f479eb13f rmse: 5.618


### Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

Ans: **5.567**

In [13]:
!pygmentize ./scripts/register_model.py

import os
import pickle
import click
import mlflow

from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models"
RF_PARAMS = ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'random_state']

mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.sklearn.autolog()


def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def train_and_log_model(data_path, params):
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

    with mlflow.start_run():
        new_params = {}
        for param in RF_PARAMS:
            new_params

In [14]:
!python ./scripts/register_model.py

2024/10/27 14:41:32 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2024/10/27 14:41:33 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/workspaces/mlops-zoomcamp-hws/02-experiment tracking/venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/workspaces/mlops-zoomcamp-hws/02-experiment tracking/venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/10/27 14:41:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run ca